In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from torchsummary import summary

In [2]:
checkpoint = torch.load('openfieldModel.pt', map_location='cpu', weights_only=True)
if isinstance(checkpoint, dict):
    print("가중치만 포함")
    for layer_name in checkpoint.keys():
        print(f"Layer: {layer_name}") # 레이어 출력
else:
    print("구조, 가중치 포함")
    print("모델 객체:", checkpoint)

가중치만 포함
Layer: epoch
Layer: model_state_dict
Layer: optimizer_state_dict
Layer: min_train_loss
Layer: min_valid_loss


## ResNet-50 신경망을 사용 하는듯

In [28]:
# 출력결과 ouput = 3
model = models.resnet50(num_classes=3)

In [4]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [5]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## 모델 요약 정보

In [6]:
device = torch.device('cpu')
summary(model, (3,244,244), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]           4,096
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]          16,384
      BatchNorm2d-12          [-1, 256, 61, 61]             512
           Conv2d-13          [-1, 256, 61, 61]          16,384
      BatchNorm2d-14          [-1, 256,

## 모델 테스트

In [44]:
# 이미지 사이즈 변환 -> Tensor -> ImageNet 정규화
transform = transforms.Compose([
    transforms.Resize(256),  # Adjust size if needed
    transforms.CenterCrop(244),
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [45]:
# 이미지 로드
image = Image.open('normal3.jpg').convert('RGB')    # 이미지 -> RGB
input_tensor = transform(image)                     # 전처리 적용
input_batch = input_tensor.unsqueeze(0)             # 배치 차원 추가
print(input_batch.size())

torch.Size([1, 3, 244, 244])


In [46]:
# 경량화 모드로 전환 후 예측
with torch.no_grad():
    output = model(input_batch)
    print(output)
    # 예측 결과를 클래스 인덱스로 변환
    _, predicted = torch.max(output, 1)

# 0, 1, 2 중 하나의 결과 출력
# 0 -> 정상 , 1 -> 질병 , 2 - > 해충
print(f'Predicted class index: {predicted.item()}')

tensor([[ 2.0269, -0.8890, -1.1205]])
Predicted class index: 0
